In [1]:
from datasets.CSVDataset import CSVDataset
from utils import reduce_dataset
from train import kfold_grid_search
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = CSVDataset("data/features/Resnet50Model_features.csv")
dataset = reduce_dataset(dataset, discard_ratio=0.0)
print(f"{len(dataset):,} examples")
kfold_grid_search(dataset, in_features=2048, device=device,
                  checkpoint_file_path=None,  #"grid-search-25.json",
                  batch_size=256,
                  max_epochs=20,
                  hidden_layer_combs=[1,2,3],
                  unit_combs=[512,1024,2048],
                  learning_rate_combs=[0.0001],
                  dropout_combs=[0.2,0.3,0.4],
                  threshold_combs=[0.5],
                  focal_alpha_combs=[0.02, 0.03, 0.04],
                  )


data/features/Resnet50Model_features.pickle
20,227 examples
(0/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.02, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 513.70it/s]


{'test_loss': 0.002544889607816003, 'test_accuracy': 0.7880847303827256, 'test_precision': 0.164004433762913, 'test_recall': 0.8223348277324671, 'test_f1': 0.27083623797024126, 'test_mcc': 0.30277693044051646}
(1/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.03, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 1025.94it/s]


{'test_loss': 0.0035857811220921577, 'test_accuracy': 0.7966721372358216, 'test_precision': 0.16841571545326603, 'test_recall': 0.8093762597985912, 'test_f1': 0.27542504462373885, 'test_mcc': 0.3051535135823206}
(2/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.04, focal_gamma=2.0)


...

{'test_loss': 0.004563293313549366, 'test_accuracy': 0.7975413911725966, 'test_precision': 0.17788649200406748, 'test_recall': 0.8085518312419435, 'test_f1': 0.2842400651711342, 'test_mcc': 0.3128157308903201}
(79/81) (hidden_layers=3, units=2048, dropout=0.4, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.03, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 256.00it/s]


{'test_loss': 0.007636663090670482, 'test_accuracy': 0.7931331700102119, 'test_precision': 0.16827107032045513, 'test_recall': 0.8220557332781597, 'test_f1': 0.2766837955303799, 'test_mcc': 0.3087364462675651}
(80/81) (hidden_layers=3, units=2048, dropout=0.4, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.04, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 256.00it/s]

{'test_loss': 0.010451708090840839, 'test_accuracy': 0.7876624058356025, 'test_precision': 0.16983857189789486, 'test_recall': 0.844731496036643, 'test_f1': 0.2785954219296909, 'test_mcc': 0.3139618712020642}

Best params: (hidden_layers=1, units=1024, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.02, focal_gamma=2.0)
Best performance: {'test_loss': 0.002569286967627704, 'test_accuracy': 0.8134816915921738, 'test_precision': 0.18573951728295565, 'test_recall': 0.8382208622238615, 'test_f1': 0.30014252209588754, 'test_mcc': 0.33490905574286917}
